### Clean and Transform Spotify Data
- Takes data from each sheet in the SpotifyPlaylistData excel file and creates a dataframe.
- Creates the genre column to merge the playlists.

In [ ]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import time

SpotifyExcelFile = '../Spotify/SpotifyPlaylistData_20230109.xlsx'

def cleanPlaylistData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)

  df = df.drop(columns=['Unnamed: 0', 'duration_ms'])

  df = df.loc[:,['track_uri','artist_uri', 'album_uri', 'track_name', 'artist_name', 'album_name', 'release_date', 'explicit', 'popularity']]
  # df['duration_ms'] = pd.to_timedelta(df['duration_ms'], unit='ms')
  df = df.reset_index(drop=True)
  
  return df

def cleanArtistData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df = df.reset_index(drop=True)
  
  return df

def cleanAlbumData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0', 'artist_uri'])
  df = df.loc[:,['album_uri', 'album_name', 'artist_name', 'release_date', 'label', 'total_tracks', 'popularity']]
  df = df.reset_index(drop=True)
  
  return df

def cleanTrackFeaturesData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df.rename(columns={'uri': 'track_uri'}, inplace=True)
  df = df.reset_index(drop=True)
  
  return df


def mergeDataFrames(HHDF, CHHDF):
  mergedDF = pd.concat([HHDF, CHHDF], ignore_index=True)

  return mergedDF

HHPlaylistSpotDF = cleanPlaylistData('HHPlaylist')
CHHPlaylistSpotDF = cleanPlaylistData('CHHPlaylist')
HHPlaylistSpotDF.insert(7, 'genre', 'Hip-Hop')
CHHPlaylistSpotDF.insert(7, 'genre', 'Christian Hip-Hop')

PlaylistSpotDF = mergeDataFrames(HHPlaylistSpotDF, CHHPlaylistSpotDF)

HHArtistsSpotDF = cleanArtistData('HHArtists')
CHHArtistsSpotDF = cleanArtistData('CHHArtists')

ArtistsSpotDF = mergeDataFrames(HHArtistsSpotDF, CHHArtistsSpotDF)

HHAlbumsSpotDF = cleanAlbumData('HHAlbums')
CHHAlbumsSpotDF = cleanAlbumData('CHHAlbums')

AlbumsSpotDF = mergeDataFrames(HHAlbumsSpotDF, CHHAlbumsSpotDF)

HHTrackFeaturesSpotDF = cleanTrackFeaturesData('HHTrackFeatures')
CHHTrackFeaturesSpotDF = cleanTrackFeaturesData('CHHTrackFeatures')

TrackFeaturesSpotDF = mergeDataFrames(HHTrackFeaturesSpotDF, CHHTrackFeaturesSpotDF)

### Clean and Transform LastFM Data
- Takes data from each sheet in the LastFMData excel file and creates a dataframe.

In [ ]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import time

LastFMExcelFile = '../LastFM/LastFMData_20230109.xlsx'

def cleanLastFMTrackData(sheetName):

  df = pd.read_excel(LastFMExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df['artist_mbid'] = df['artist_mbid'].replace(np.nan, None)
  df = df.reset_index(drop=True)
  
  return df

def cleanLastFMArtistData(sheetName):

  df = pd.read_excel(LastFMExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df = df.drop_duplicates(['artist_name'])
  df['artist_mbid'] = df['artist_mbid'].replace(np.nan, None)
  df = df.reset_index(drop=True)
  
  return df

def cleanLastFMAlbumData(sheetName):

  df = pd.read_excel(LastFMExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df = df.drop_duplicates(subset=['album_mbid', 'album_name'])
  df['album_mbid'] = df['album_mbid'].replace(np.nan, None)
  df = df.reset_index(drop=True)
  
  return df

def mergeDataFrames(HHDF, CHHDF):
  mergedDF = pd.concat([HHDF, CHHDF], ignore_index=True)

  return mergedDF

HHAlbumLastDF = cleanLastFMAlbumData('HHAlbumData')
CHHAlbumLastDF = cleanLastFMAlbumData('CHHAlbumData')
AlbumsLastDF = mergeDataFrames(HHAlbumLastDF, CHHAlbumLastDF)

HHTrackLastDF = cleanLastFMTrackData('HHTrackData')
CHHTrackLastDF = cleanLastFMTrackData('CHHTrackData')
TrackLastDF = mergeDataFrames(HHTrackLastDF, CHHTrackLastDF)

HHArtistLastDF = cleanLastFMArtistData('HHArtistData')
CHHArtistLastDF = cleanLastFMArtistData('CHHArtistData')
ArtistLastDF = mergeDataFrames(HHArtistLastDF, CHHArtistLastDF)

# HHArtistLastDF.head()
# HHAlbumLastDF.head()
# TrackLastDF.head()

### Join Spotify and LastFM Data

In [ ]:
joinedPlaylisttDF = PlaylistSpotDF.join(TrackLastDF, lsuffix='_Spotify', rsuffix='_LastFM')
joinedPlaylisttDF = joinedPlaylisttDF.drop(columns=['artist_mbid', 'artist_name_LastFM', 'track_name_LastFM', 'album_name_LastFM'])
joinedPlaylisttDF.rename(columns={'track_name_Spotify': 'track_name', 'artist_name_Spotify': 'artist_name', 'album_name_Spotify': 'album_name', 'listeners': 'listeners_LastFM', 'playcount': 'playcount_LastFM'}, inplace=True)

joinedArtistsDF = ArtistsSpotDF.join(ArtistLastDF, lsuffix='_Spotify', rsuffix='_LastFM')
joinedArtistsDF = joinedArtistsDF.drop(columns=['artist_mbid', 'artist_name_LastFM'])
joinedArtistsDF.rename(columns={'artist_name_Spotify': 'artist_name', 'listeners': 'listeners_LastFM', 'playcount': 'playcount_LastFM'}, inplace=True)

joinedAlbumsDF = AlbumsSpotDF.join(AlbumsLastDF, lsuffix='_Spotify', rsuffix='_LastFM')
joinedAlbumsDF = joinedAlbumsDF.drop(columns=['album_mbid', 'album_name_LastFM'])
joinedAlbumsDF.rename(columns={'album_name_Spotify': 'album_name', 'listeners': 'listeners_LastFM', 'playcount': 'playcount_LastFM'}, inplace=True)

### Create Excel File
- Compiles the json files created from each API call into TransformedMusicData excel file.
- The file is used in the Tableau project TrackStarz Music Analysis:
https://public.tableau.com/app/profile/andrew.grant6857/viz/TrackStarzMusicAnalysis

In [ ]:
# Push data to excel

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import datetime as datetime

xlxWriter = pd.ExcelWriter('TransformedMusicData.xlsx', engine='xlsxwriter')

joinedPlaylisttDF.to_excel(xlxWriter, sheet_name='Playlist')
joinedArtistsDF.to_excel(xlxWriter, sheet_name='Artists')
joinedAlbumsDF.to_excel(xlxWriter, sheet_name='Albums')
TrackFeaturesSpotDF.to_excel(xlxWriter, sheet_name='TrackFeatures')

xlxWriter.close()

### Move Data to SQL Server
- Uses local MSSQL server to store the data.
- Creates a table for each dataframe.

In [ ]:
import pyodbc
import sqlalchemy
from sqlalchemy.engine import URL
from sqlalchemy.types import SmallInteger, Text, String, DateTime, Boolean, Integer, Float
import os
from dotenv import load_dotenv
import pandas as pd
import json

load_dotenv()

LOCAL_SERVER_NAME = os.getenv('LOCAL_SERVER_NAME')
dbName = 'TrackStarzMusicAnalysis'

connection_string = ("Driver={SQL Server Native Client 11.0};"
            f"Server={LOCAL_SERVER_NAME};"
            f"Database={dbName};"
            "Trusted_Connection=yes")
      
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = sqlalchemy.create_engine(connection_url)

conn = engine.connect()

def createSQLTable(dataFrame, tableName, dataTypes):
  dataFrame.to_sql(tableName, 
    con=conn, 
    if_exists='replace', 
    index=False, 
    dtype=dataTypes)

createSQLTable(joinedPlaylisttDF, 'SpotifyPlaylist', {
    "track_uri": String(50),
    "artist_uri": String(50),
    "album_uri": String(50),
    "track_name": String(100),
    "artist_name": String(100),
    "album_name": String(100),
    "release_date": DateTime(),
    "explicit": SmallInteger(),
    "popularity": SmallInteger(),
    "listeners_LastFM": Integer(),
    "playcount_LastFM": Integer()
})

createSQLTable(joinedArtistsDF, 'SpotifyArtists', {
    "artist_uri": String(50),
    "artist_name": String(100),
    "popularity": SmallInteger(),
    "followers": Integer(),
    "listeners_LastFM": Integer(),
    "playcount_LastFM": Integer()
})

conn.execute('ALTER TABLE SpotifyArtists ALTER COLUMN artist_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyArtists ADD PRIMARY KEY (artist_uri);')

createSQLTable(joinedAlbumsDF, 'SpotifyAlbums', {
    "album_uri": String(50),
    "album_name": String(100),
    "artist_name": String(100),
    "release_date": DateTime(),
    "label": String(100),
    "total_tracks": SmallInteger(),
    "popularity": SmallInteger(),
    "listeners_LastFM": Integer(),
    "playcount_LastFM": Integer()
})

conn.execute('ALTER TABLE SpotifyAlbums ALTER COLUMN album_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyAlbums ADD PRIMARY KEY (album_uri);')

createSQLTable(TrackFeaturesSpotDF, 'SpotifyTrackFeatures', {
    "track_uri": String(50),
    "track_name": String(100),
    "duration_ms": Integer(),
    "time_signature": SmallInteger(),
    "tempo": Float(),
    "valence": Float(),
    "liveness": Float(),
    "instrumentalness": Float(),
    "acousticness": Float(),
    "speechiness": Float(),
    "mode": SmallInteger(),
    "loudness": Float(),
    "key": SmallInteger(),
    "energy": Float(),
    "danceability": Float()
})

conn.execute('ALTER TABLE SpotifyTrackFeatures ALTER COLUMN track_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyTrackFeatures ADD PRIMARY KEY (track_uri);')


conn.execute('ALTER TABLE SpotifyPlaylist ALTER COLUMN track_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyPlaylist ADD PRIMARY KEY (track_uri);')
conn.execute('ALTER TABLE SpotifyPlaylist ALTER COLUMN artist_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyPlaylist ADD FOREIGN KEY (artist_uri) REFERENCES SpotifyArtists(artist_uri);')
conn.execute('ALTER TABLE SpotifyPlaylist ALTER COLUMN album_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyPlaylist ADD FOREIGN KEY (album_uri) REFERENCES SpotifyAlbums(album_uri);')

conn.close()